In [ ]:
!pip install ../input/pathfinder/python_box-5.4.1-py3-none-any.whl
!pip install ../input/timm0412/timm-0.4.12-py3-none-any.whl

In [ ]:
import os
import gc
import sys
import warnings
import numpy as np
import pandas as pd
from box import Box
from timm import create_model
import cv2

import torch
import torch.nn as nn
from torchvision.io import read_image
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
import pytorch_lightning as pl

warnings.filterwarnings("ignore")

In [ ]:
config = {'seed': 2021,
          'root': '/kaggle/input/petfinder-pawpularity-score/', 
          'transform':{
              'name': 'get_default_transforms',
              'image_size': 224
          },
          'dataloader':{
              'batch_size': 64,
              'shuffle': False,
              'num_workers': 4,
              'pin_memory': True,
              'drop_last': False,
          },
          'model':{
              'name': 'swin_large_patch4_window7_224_in22k',
              'freeze_backbone': False,
              'output_dim': 1
          },
          'trainer':{
              'gpus': 1
          },
          'test':{
              'n_fold': 5,
              'trn_folds': [0, 1, 2, 3, 4],
              'tta_steps': 5
          }
}

config = Box(config)

In [ ]:
class PetfinderDataset(Dataset):
    def __init__(self, df):
        self._X = df["Id"].values
        self.transform = get_default_transforms()

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = read_image(image_path)
        image = self.transform(image)
        return image

In [ ]:
def get_default_transforms():
    transform = T.Compose(
            [
                T.Resize([config.transform.image_size, config.transform.image_size]),
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=[0.485, 0.456, 0.406], 
                            std=[0.229, 0.224, 0.225]),
            ]
        )
    return transform

In [ ]:
class Model(pl.LightningModule):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.__build_model()
        self.save_hyperparameters(cfg)

    def __build_model(self):
        self.backbone = create_model(
            config.model.name, pretrained=False, num_classes=0, in_chans=3
        )
        num_features = self.backbone.num_features
        self.fc = nn.Sequential(
            nn.Linear(num_features, 384),
            nn.Dropout(0.1),
            nn.Linear(384, 96), 
            nn.Linear(96, config.model.output_dim)
        )
        
    def forward(self, x):
        f = self.backbone(x)
        out = self.fc(f)
        return out

    def predict_step(self, batch, batch_idx):
        x = batch
        out = self(x)
        return out.sigmoid()

In [ ]:
def get_weights_path_224(fold):
    path = "../input/v4drop01"
    return os.path.join(path, f"best_loss_v{fold}.ckpt")

def get_weights_path_384(fold):
    path = "../input/swin384drop01"
    return os.path.join(path, f"best_loss_v{fold}.ckpt")

In [ ]:
df = pd.read_csv(os.path.join(config.root, "test.csv"))
df["Id"] = df["Id"].apply(lambda x: os.path.join(config.root, "test", x + ".jpg"))

In [ ]:
models = ['swin_large_patch4_window7_224_in22k',
             'swin_large_patch4_window12_384_in22k']
image_sizes = [224, 384]
opt_weights=[0.095, 0.095, 0.095, 0.095, 0.095, 0.105, 0.105, 0.105, 0.105, 0.105]

def get_fold_predictions(fold):
    tta_preds = []
    for m, img_size in zip(models, image_sizes):
        config.model.name = m
        config.transform.image_size = img_size
        if img_size == 224:
            weight = get_weights_path_224(fold)
        else:
            weight = get_weights_path_384(fold)
        model = Model.load_from_checkpoint(checkpoint_path=weight, cfg=config)
        trainer = pl.Trainer(**config.trainer)
        
        for tta in range(config.test.tta_steps):
            ds = PetfinderDataset(df)
            dl = DataLoader(ds, **config.dataloader)
            predictions = trainer.predict(model, dl)
            tta_preds.append(torch.cat([x * 100 for x in predictions]).detach().cpu().numpy())
        
        del model; gc.collect()
        torch.cuda.empty_cache()
        
#     predictions = np.mean(np.column_stack(tta_preds), axis=1)
    predictions = np.average(np.column_stack(tta_preds), axis=1, weights=opt_weights)
    return predictions

In [ ]:
fold_predictions = []
for fold in config.test.trn_folds:
    preds = get_fold_predictions(fold)
    fold_predictions.append(preds)
    
final_predictions = np.mean(np.column_stack(fold_predictions), axis=1)

In [ ]:
submission = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")
submission["Pawpularity"] = final_predictions
submission.to_csv("submission.csv", index=False)
submission.head()